In [1]:
# from TurkishStemmer import TurkishStemmer
# stemmer = TurkishStemmer()

# a = ("bakkaldan eve dönerken babamı gördüm de ne oldu bir tahmin eder misin?")

# a = [(stemmer.stem(nltk.word_tokenize(a.lower())))]
# print(a)
# print(turkStem.stemWord('gördüm'))


In [2]:
import nltk 
# from nltk.stem.lancaster import LancasterStemmer
# nltk.download('punkt')
import time 
# stemmer = LancasterStemmer()

import numpy 
import tflearn 
import tensorflow 
from tensorflow.python.framework import ops

import random 
import json 
import pickle
from snowballstemmer import TurkishStemmer
turkStem = TurkishStemmer()

# with open('my_intents2.json') as file:
#     data = json.load(file)
#     print("merhabaaaaaaaaa")

with open('intents_.json') as file:
        data = json.load(file)
        print("merhabaaaaaaaaa")

try: 
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except: 
    print('saaaaa')
    words = []
    labels = []
    docs_x = []
    docs_y = [] #2.part

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            
            words.extend(wrds) #iki diziyi birleştirir.
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])     
            
    words = [turkStem.stemWord(w.lower()) for w in words if w != "?"] # tüm pattern'ın içerisindeki kelimeleri parçalayıp attık.
    words = sorted(list(set(words)))
    
    labels = sorted(labels)
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]


# until 2-part
for x, doc in enumerate(docs_x):
    bag = []

    wrds = [turkStem.stemWord(w.lower()) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)
    

training = numpy.array(training)
output = numpy.array(output)

# with open("data.pickle","wb") as f:
#     pickle.dump((words, labels, training, output), f)

# 3-part
ops.reset_default_graph()
# tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


# model.load("model.tflearn")
#batch_size : girdilerin sayısı
model.fit(training, output, n_epoch=1000, batch_size=90, show_metric=True)
model.save("model.tflearn")


Training Step: 3999  | total loss: 1.39236 | time: 0.015s
| Adam | epoch: 1000 | loss: 1.39236 - acc: 0.6881 -- iter: 270/348
Training Step: 4000  | total loss: 1.36096 | time: 0.027s
| Adam | epoch: 1000 | loss: 1.36096 - acc: 0.6834 -- iter: 348/348
--
INFO:tensorflow:/home/yazilimci/Documents/chatbot/14aralık_guncelleme/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [turkStem.stemWord(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

import pyttsx3

def chat():
    print("Merhaba benim adım Gazi:) \nÜniversitemiz hakkında merak ettiklerini bana sorabilirsin.")
    while True:
        inp = input("Sen: ")
        if inp.lower() == "quit":
            break
        results = model.predict([bag_of_words(inp, words)])
        
        results_index = numpy.argmax(results)
        print('********************\n')
        tag = labels[results_index]
        print("TAGG ==== ",tag)
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
        print("Bu konuyu mu aramıştınız: ", tag)
        print()
        inp2 = input("evetse 1 e hayır ise 2'ye basınız : ")
        if inp2 == "1":
            print(responses)
        else :
            print("Doğru sözcükler ile tekrar deneyin")

chat()

Merhaba benim adım Gazi:) 
Üniversitemiz hakkında merak ettiğin bir konu varsa bana sorabilirsin.
Sen: sınav takvimi
********************

TAGG ====  akademik takvim
Bunu mu aradınız?:  akademik takvim

Evet ise 1, hayır ise 2'ye basınız: 1
http://gazi-universitesi.gazi.edu.tr/posts?type=akademiktakvim
Sen: öğrenci istatistik
********************

TAGG ====  öğrenci i̇şleri daire başkanlığı
Bu konuyu mu aramıştınız:  öğrenci i̇şleri daire başkanlığı

evetse 1 e hayır ise 2'ye basınız : personel istatiktikliği
Doğru sözcüklerle tekrar deneyiniz


In [ ]:
#part-4


